Step 1:  Install and import needed packages and libraries, set active session

In [ ]:
# Import python packages, added numpy, sckit-learn and pandas
import streamlit as st
import pandas as pd
import numpy as np
import pickle
import gzip
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from snowflake.snowpark.types import ArrayType, DoubleType

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


Step 2:  Import train_table into Pandas DataFrame.  Set features for X and label for y.  Establish train/test split

In [ ]:

data = session.table("demo_db.streaming.train_table").to_pandas()

X = data[['FEED_RATE_ABNORMAL_PCT', 'FEED_RATE_AVG', 'VIBRATION_ABNORMAL_PCT', 'VIBRATION_AVG', 'SPINDLE_SPEED_ABNORMAL_PCT', 'SPINDLE_SPEED_AVG', 'TOOL_WEAR']]
y = data['QUALITY_YIELD']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Step 3:  Create a class that limits the predicted value to a maximum of 1, create a LinearRegression model and write to model.pkl

In [ ]:
class ClippedLinearRegression(LinearRegression):
    def predict(self, X):
        predictions = super().predict(X)
        return np.clip(predictions, 0, 1)

# Create and train the model
model = ClippedLinearRegression()
model.fit(X_train, y_train)

# Save the model
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

Step 4:  Save the model.pkl file to @my_stage

In [ ]:
session.file.put("model.pkl", "@my_stage", overwrite=True)

Step 5:  Unzip model.pkl.gz, use it to create and register an PREDICT_MODEL UDF

In [ ]:


# Define a local directory to download the file to (for example, '/tmp')
target_directory = '/tmp'

# Download the model file to the local directory
session.file.get("@my_stage/model.pkl.gz", target_directory)

# Now, the file is available locally at /tmp/model.pkl.gz
model_file_path = target_directory + "/model.pkl.gz"

# Open the file and decompress it
with gzip.GzipFile(model_file_path, mode='rb') as f:
    model = pickle.load(f)



# Define the UDF function without the decorator
def predict_udf(features):
    # Predict based on features
    prediction = model.predict([features])[0]
    return prediction

# Specify the stage location where your UDF code will be stored
stage_location = "@my_stage"  # Replace this with your actual stage location

# Register UDF expecting an array of Doubles
session.udf.register(
    predict_udf, 
    name="PREDICT_MODEL", 
    input_types=[ArrayType(DoubleType())],  # Specify array input type
    return_type=DoubleType(),  # Specify return type
    is_permanent=True, 
    replace=True, 
    stage_location=stage_location,
    packages=["scikit-learn"]
)


Step 6:  Run the model and compare predictions against the train_table

In [ ]:
SELECT 
    BATCH_ID, 
    MACHINE_NAME,
    FEED_RATE_ABNORMAL_PCT,
    FEED_RATE_AVG,
    VIBRATION_ABNORMAL_PCT, 
    VIBRATION_AVG, 
    SPINDLE_SPEED_ABNORMAL_PCT,
    SPINDLE_SPEED_AVG,
    TOOL_WEAR,
    QUALITY_YIELD,
    PREDICT_MODEL(
        ARRAY_CONSTRUCT(
            FEED_RATE_ABNORMAL_PCT,
            FEED_RATE_AVG,
            VIBRATION_ABNORMAL_PCT, 
            VIBRATION_AVG, 
            SPINDLE_SPEED_ABNORMAL_PCT,
            SPINDLE_SPEED_AVG,
            TOOL_WEAR
        )
    ) AS predicted_value
FROM train_table;

Step 7:  Run the PREDICT_MODEL UDF on top of the machine_stats view for near real time prediction

In [ ]:
SELECT 
    MACHINE_NAME,
    FEED_RATE_ABNORMAL_PCT,
    FEED_RATE_AVG,
    VIBRATION_ABNORMAL_PCT, 
    VIBRATION_AVG, 
    SPINDLE_SPEED_ABNORMAL_PCT,
    SPINDLE_SPEED_AVG,
    TOOL_WEAR,
    PREDICT_MODEL(
        ARRAY_CONSTRUCT(
            FEED_RATE_ABNORMAL_PCT,
            FEED_RATE_AVG,
            VIBRATION_ABNORMAL_PCT, 
            VIBRATION_AVG, 
            SPINDLE_SPEED_ABNORMAL_PCT,
            SPINDLE_SPEED_AVG,
            TOOL_WEAR
        )
    ) AS PREDICTED_YIELD
FROM MACHINE_STATS;

Step 8:  Evaluate model metrics

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, root_mean_squared_error

# Generate predictions on the test set
y_pred = model.predict(X_test)

# Clip the predictions if needed
y_pred_clipped = np.clip(y_pred, 0, 1)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred_clipped)
mse = mean_squared_error(y_test, y_pred_clipped)
rmse = root_mean_squared_error(y_test, y_pred_clipped)
r2 = r2_score(y_test, y_pred_clipped)

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R-squared (R²): {r2:.4f}")

Mean Absolute Error (MAE) - Measures avg prediction error in same units as label.  Lower is better, ideally <0.1  

Mean Squared Error (MSE) - Avg squared dif between actual and predicted values.  Lower is better.  <0.01 Good fit, 0.01 to 0.05 Acceptable, MSE > 0.05 model needs improvement

Root Mean Squared Error (RMSE) - Lower is better, typically <0.1 meaning average prediction error is within 10 percentage points  

R-Squared - How ell the independaent variables explain the variance.  (Ranges from 0 to 1) Higher is better.  >0.7 considered strong, >0.5 is acceptable, and <0.3 is weak